MLFlow Code Here. All the above is a step by step walk through, while the code below should be ran using MLFlow and the expirement we set up.

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import datetime as dt
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [3]:
%scala
val tags = com.databricks.logging.AttributionContext.current.tags
val username = tags.getOrElse(com.databricks.logging.BaseTagDefinitions.TAG_USER, java.util.UUID.randomUUID.toString.replace("-", ""))
spark.conf.set("com.databricks.demo.username", username)

tags: Map[com.databricks.logging.TagDefinition,String] = Map(TagDefinition(opId,) -> HttpServer-86992dd2f73111f3, TagDefinition(opTarget,) -> /notebook/168519256284520/command/168519256284556, TagDefinition(clusterMemory,) -> 18432, TagDefinition(notebookId,) -> 168519256284520, TagDefinition(projectName,) -> webapp, TagDefinition(eventWindowTime,) -> 8466893.15, TagDefinition(httpTarget,) -> /notebook/168519256284520/command/168519256284556, TagDefinition(buildHash,) -> "", TagDefinition(browserHash,) -> #notebook/168519256284520/command/168519256284553, TagDefinition(host,Host where the request is coming from.) -> 10.139.64.5, TagDefinition(notebookLanguage,) -> python, TagDefinition(sparkVersion,) -> 5.3.x-scala2.11, TagDefinition(hostName,) -> cons-webapp-1, TagDefinition(httpMethod,) -> POST, TagDefinition(browserIdleTime,) -> 1200, TagDefinition(browserTabId,) -> 4ae3e646-b6b1-4c8b-a192-56f280f998f7, TagDefinition(sourceIpAddress,) -> 73.193.17.210, TagDefinition(browserUserAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36, TagDefinition(orgId,) -> 1559977908724754, TagDefinition(userAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36, TagDefinition(clusterId,) -> 0703-220934-empty762, TagDefinition(rootOpId,Unique identifier for a root of a trace (tree of ops).) -> 8017431669755007196_5592691618263308068_ce5a0eb6e0884288aecedb71552d6152, TagDefinition(sessionId,) -> 9ce6b1371cac4b995920a0619bc7761bd178b8d8873bfb14e30ce9a9dacd64b7, TagDefinition(clusterCreator,) -> Webapp, TagDefinition(clientBranchName,) -> 2.100.1036, TagDefinition(clusterType,) -> spot, TagDefinition(browserHasFocus,) -> true, TagDefinition(userId,) -> 8499602646880646, TagDefinition(browserIsHidden,) -> false, TagDefinition(opType,) -> HttpServer, TagDefinition(user,) -> ryan.chynoweth@insight.com, TagDefinition(browserHostName,) -> westus.azuredatabricks.net, TagDefinition(parentOpId,) -> ServiceMain-86992dd2d7310002, TagDefinition(jettyRpcType,) -> "com.databricks.backend.common.rpc.InternalDriverBackendMessages$DriverBackendRequest")
username: String = ryan.chynoweth@insight.com

In [4]:
client = MlflowClient()
exps = client.list_experiments()
exp = [s for s in exps if "/Users/{}/BikeSharing".format(spark.conf.get("com.databricks.demo.username")) in s.name][0]
exp_id = exp.experiment_id
artifact_location = exp.artifact_location
run = client.create_run(exp_id)
run_id = run.info.run_id 

In [5]:
mlflow.start_run(run_id)

Out[ 6 ]: <ActiveRun: >

In [6]:
try: 
  # load data
  df = spark.read.format('delta').load("/mnt/delta/silver/bikeSharing/hourly")
  
  # split data
  train_df, test_df = df.randomSplit([0.7, 0.3])

  # One Hot Encoding
  mnth_encoder = OneHotEncoder(inputCol="mnth", outputCol="encoded_mnth")
  hr_encoder = OneHotEncoder(inputCol="hr", outputCol="encoded_hr")
  weekday_encoder = OneHotEncoder(inputCol="weekday", outputCol="encoded_weekday")

  # set the training variables we want to use
  train_cols = ['encoded_mnth', 'encoded_hr', 'encoded_weekday', 'temp', 'hum']

  # convert cols to a single features col
  assembler = VectorAssembler(inputCols=train_cols, outputCol="features")

  # Set linear regression model
  lr = LinearRegression(featuresCol="features", labelCol="cnt")

  # Create pipeline
  pipeline = Pipeline(stages=[
    mnth_encoder,
    hr_encoder,
    weekday_encoder,
    assembler,
    lr
  ])

  # fit pipeline
  lrPipelineModel = pipeline.fit(train_df)

  # write model to datetime folder and latest folder
  lrPipelineModel.write().overwrite().save("{}/latest/bike_sharing_model.model".format(artifact_location))
  lrPipelineModel.write().overwrite().save("{}/year={}/month={}/day={}/bike_sharing_model.model".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))

  # write test predictions to datetime and lastest folder
  predictions = lrPipelineModel.transform(test_df)
  predictions.write.format("parquet").mode("overwrite").save("{}/latest/test_predictions.parquet".format(artifact_location))
  predictions.write.format("parquet").mode("overwrite").save("{}/year={}/month={}/day={}/test_predictions.parquet".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))

  # mlflow log evaluations
  evaluator = RegressionEvaluator(labelCol = "cnt", predictionCol = "prediction")

  mlflow.log_metric("mae", evaluator.evaluate(predictions, {evaluator.metricName: "mae"}))
  mlflow.log_metric("rmse", evaluator.evaluate(predictions, {evaluator.metricName: "rmse"}))
  mlflow.log_metric("r2", evaluator.evaluate(predictions, {evaluator.metricName: "r2"}))
  mlflow.set_tag("Model Path", "{}/year={}/month={}/day={}".format(artifact_location, dt.datetime.utcnow().year, dt.datetime.utcnow().month, dt.datetime.utcnow().day))
  
  mlflow.end_run(status="FINISHED")
  print("Model training finished successfully")
except Exception as e:
    mlflow.log_param("Error", str(e))
    mlflow.end_run(status="FAILED")
    print("Model training failed: {}".format(str(e)))


Model training finished successfully